# Run from here if you don't have the network_dataset.csv file

In [1]:
import polars as pl

df1_nul = pl.read_csv(
    r"dataset\Network datatset\csv\attack_1.csv",
    encoding="utf-8",
    separator=",",
    ignore_errors=True,
    null_values=["N/A"]
)

df2_nul = pl.read_csv(
    r"dataset\Network datatset\csv\attack_2.csv",
    encoding="utf-8",
    separator=",",
    ignore_errors=True,
    null_values=["N/A"]
)

df3_nul = pl.read_csv(
    r"dataset\Network datatset\csv\attack_3.csv",
    encoding="utf-8",
    separator=",",
    ignore_errors=True,
    null_values=["N/A"]
)

df4_nul = pl.read_csv(
    r"dataset\Network datatset\csv\attack_4.csv",
    encoding="utf-8",
    separator=",",
    ignore_errors=True,
    null_values=["N/A"]
)

df5_nul = pl.read_csv(
    r"dataset\Network datatset\csv\normal.csv",
    encoding="utf-8",
    separator=",",
    ignore_errors=True,
    null_values=["N/A"]
)

In [2]:
# update all columns using by removing leading and trailing spaces
df_list = [df1_nul, df2_nul, df3_nul, df4_nul, df5_nul]
for i, df in enumerate(df_list):
    df_list[i] = df.rename(mapping={col: col.strip() for col in df.columns})
df1, df2, df3, df4, df5 = df_list

In [3]:
print(df1.columns)
print(df4.columns)

right_order = df1.columns
df4 = df4.select(right_order)


['Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'proto', 'flags', 'size', 'modbus_fn', 'n_pkt_src', 'n_pkt_dst', 'modbus_response', 'label_n', 'label']
['Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'proto', 'flags', 'size', 'modbus_fn', 'modbus_response', 'n_pkt_src', 'n_pkt_dst', 'label_n', 'label']


In [4]:
# concat the dataframes vertically
df = pl.concat([df1, df2, df3, df4, df5], how="vertical")
df.write_csv("dataset/Network datatset/csv/network_dataset.csv")

# Run from here if you already have the network_dataset.csv file

In [2]:
import polars as pl
df = pl.read_csv("dataset/Network datatset/csv/network_dataset.csv")
df.head()


Time,mac_s,mac_d,ip_s,ip_d,sport,dport,proto,flags,size,modbus_fn,n_pkt_src,n_pkt_dst,modbus_response,label_n,label
str,str,str,str,str,i64,i64,str,i64,i64,str,i64,i64,str,i64,str
"""2021-04-09 18:23:28.385003""","""74:46:a0:bd:a7:1b""","""0a:fe:ec:47:74:fb""","""84.3.251.20""","""84.3.251.102""",56667,502,"""Modbus""",11000,66,"""Read Coils Request""",0,0,null,0,"""normal"""
"""2021-04-09 18:23:28.385005""","""74:46:a0:bd:a7:1b""","""e6:3f:ac:c9:a8:8c""","""84.3.251.20""","""84.3.251.101""",56666,502,"""Modbus""",11000,66,"""Read Coils Request""",1,0,null,0,"""normal"""
"""2021-04-09 18:23:28.385006""","""74:46:a0:bd:a7:1b""","""fa:00:bc:90:d7:fa""","""84.3.251.20""","""84.3.251.103""",56668,502,"""Modbus""",11000,66,"""Read Coils Request""",2,0,null,0,"""normal"""
"""2021-04-09 18:23:28.385484""","""0a:fe:ec:47:74:fb""","""74:46:a0:bd:a7:1b""","""84.3.251.102""","""84.3.251.20""",502,56667,"""Modbus""",11000,64,"""Read Coils Response""",0,0,"""[0]""",0,"""normal"""
"""2021-04-09 18:23:28.385486""","""fa:00:bc:90:d7:fa""","""74:46:a0:bd:a7:1b""","""84.3.251.103""","""84.3.251.20""",502,56668,"""Modbus""",11000,64,"""Read Coils Response""",0,1,"""[0]""",0,"""normal"""


In [3]:
df.shape

(29829204, 16)